In [ ]:
!pip uninstall torch transformers datasets wandb -y
!pip install torch transformers datasets
!pip install --upgrade torch transformers datasets


Found existing installation: torch 2.5.1
Uninstalling torch-2.5.1:
  Successfully uninstalled torch-2.5.1
Found existing installation: transformers 4.47.0
Uninstalling transformers-4.47.0:
  Successfully uninstalled transformers-4.47.0
Found existing installation: datasets 3.2.0
Uninstalling datasets-3.2.0:
  Successfully uninstalled datasets-3.2.0
  Using cached torch-2.5.1-cp310-cp310-manylinux1_x86_64.whl.metadata (28 kB)
  Using cached transformers-4.47.0-py3-none-any.whl.metadata (43 kB)
  Using cached datasets-3.2.0-py3-none-any.whl.metadata (20 kB)
Using cached torch-2.5.1-cp310-cp310-manylinux1_x86_64.whl (906.4 MB)
Using cached transformers-4.47.0-py3-none-any.whl (10.1 MB)
Using cached datasets-3.2.0-py3-none-any.whl (480 kB)


In [1]:
import shutil

# Распаковка train_dataset
shutil.unpack_archive('train_dataset.zip', 'train_dataset')

# Распаковка test_dataset
shutil.unpack_archive('test_dataset.zip', 'test_dataset')

# Распаковка токенизатора
shutil.unpack_archive('tokenizer.zip', 'tokenizer')


In [2]:
from transformers import GPT2LMHeadModel, Trainer, TrainingArguments

c:\Cove\Python Projects\GPT2_train\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from datasets import load_from_disk

# Загрузка train и test датасетов
train_dataset = load_from_disk('train_dataset')
test_dataset = load_from_disk('test_dataset')


In [4]:
from transformers import AutoTokenizer

# Загрузка токенизатора
tokenizer = AutoTokenizer.from_pretrained('tokenizer')


In [5]:
# Загрузка GPT-2
model = GPT2LMHeadModel.from_pretrained("gpt2")


In [6]:
from transformers import AutoTokenizer, GPT2LMHeadModel, Trainer, TrainingArguments, DataCollatorForLanguageModeling

In [7]:
# Подготовка DataCollator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,  # GPT-2 не использует MLM
)

In [8]:
import accelerate

accelerate.__version__

'1.2.1'

In [33]:
# Настройки обучения
training_args = TrainingArguments(
    fp16=True,  # Включение смешанной точности
    output_dir="./results",
    eval_strategy="epoch",
    learning_rate=2e-5, #увеличина скорость обучения, было 5e-5
    per_device_train_batch_size=6, #выше не дает, ругается на ограничения
    num_train_epochs=2, #снижено до 2, было 3
    save_steps=10_000,
    save_total_limit=2,
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [34]:
import os
os.environ["WANDB_DISABLED"] = "true"

# Trainer
trainer = Trainer(
    model=model.to("cuda"),  # Убедитесь, что модель на GPU
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator
)

C:\Users\Alex\AppData\Local\Temp\ipykernel_12632\4279693538.py:5: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [36]:
# Обучение модели
trainer.train()

  0%|          | 500/149592 [02:07<10:06:35,  4.10it/s]

{'loss': 1.6085, 'grad_norm': 2.3898446559906006, 'learning_rate': 1.9933285202417242e-05, 'epoch': 0.01}


  1%|          | 1000/149592 [04:12<10:02:08,  4.11it/s]

{'loss': 1.5006, 'grad_norm': 2.4327454566955566, 'learning_rate': 1.986643670784534e-05, 'epoch': 0.01}


  1%|          | 1500/149592 [06:16<10:14:40,  4.02it/s]

...

100%|█████████▉| 149500/149592 [13:54:56<00:40,  2.29it/s]

{'loss': 0.8905, 'grad_norm': 2.2147881984710693, 'learning_rate': 2.112412428472111e-08, 'epoch': 2.0}


                                                          
100%|██████████| 149592/149592 [14:15:35<00:00,  2.91it/s]

{'eval_loss': 0.8554497957229614, 'eval_runtime': 1197.1583, 'eval_samples_per_second': 41.652, 'eval_steps_per_second': 5.206, 'epoch': 2.0}
{'train_runtime': 51335.3086, 'train_samples_per_second': 17.484, 'train_steps_per_second': 2.914, 'train_loss': 0.9628696900152562, 'epoch': 2.0}


TrainOutput(global_step=149592, training_loss=0.9628696900152562, metrics={'train_runtime': 51335.3086, 'train_samples_per_second': 17.484, 'train_steps_per_second': 2.914, 'total_flos': 2.34527366578176e+17, 'train_loss': 0.9628696900152562, 'epoch': 2.0})

In [37]:
trainer.save_model("./my_model")

In [38]:
model.save_pretrained("./model")